In [1]:
with open("input.txt") as f:
    lines = [
        line.strip() for line in f.readlines()
    ]

In [2]:
from collections import namedtuple

class Range:

    def __init__(self, lower: int, upper: int):
        self.lower = lower
        self.upper = upper

    def __contains__(self, val: int):
        return self.lower <= val <= self.upper

class Rule:

    def __init__(self, name: str, range1: Range, range2: Range):
        self.name = name
        self.range1 = range1
        self.range2 = range2

    def __contains__(self, val: int):
        return val in self.range1 or val in self.range2

Ticket = namedtuple("Ticket", ["values"])

In [3]:
import re

rule_regex = "((?:\w|\s)+): (\d+)-(\d+) or (\d+)-(\d+)"
ticket_regex = "(\d+,)+(\d+)"

all_rules = []
tickets = []

for line in lines:
    if match := re.fullmatch(rule_regex, line):
        rule_name = match.group(1)
        lower1, upper1 = int(match.group(2)), int(match.group(3))
        lower2, upper2 = int(match.group(4)), int(match.group(5))

        range1 = Range(lower1, upper1)
        range2 = Range(lower2, upper2)

        rule = Rule(rule_name, range1, range2)
        all_rules.append(rule)
    elif re.fullmatch(ticket_regex, line):
        values = line.split(",")
        values = list(map(int, values))
        tickets.append(Ticket(values))

## part1

In [4]:
error_rate = 0

for ticket in tickets:
    for value in ticket.values:
        if not any(value in rule for rule in all_rules):
            error_rate += value

error_rate

30869

## part2

In [5]:
# to be a valid ticket, every value must match
# at least one rule
valid_tickets = [
    ticket
    for ticket in tickets
    if all(
        any(value in rule for rule in all_rules)
        for value in ticket.values
    )
]

Start by assuming that every rule could be valid
for every index. Then, go through all of the tickets
and all of the values in each ticket. When we come
across a value that doesn't pass a particular rule,
then it's removed as a possibility.

In [6]:
from copy import copy
from typing import List

rules_by_index: List[List[Rule]] = [copy(all_rules) for _ in all_rules]

for ticket in valid_tickets:
    for index, value in enumerate(ticket.values):
        rules_by_index[index] = [
            rule for rule in rules_by_index[index] if value in rule
        ]

After this first pass, we're left with a list that contains a
list of rules that could fit at each index. 

Now, we have to fit a specific rule to each index. If there is
only possible rule at a given index then we have our answer for
that index. Once we have our answer for an index, we remove that
rule as a possibility from every other index. We repeat this
process until we've matched a rule to every index.

In [7]:
rule_by_index: List[Rule] = [None] * len(rules_by_index)

def remove_from_possible_rules(
    rules_by_index: List[List[Rule]],
    rule_to_remove: Rule
) -> List[List[Rule]]:
    """Given a list of possible rules at each index and
    a specific rule, remove the rule as a possibility
    from each index.
    """
    return [
        [rule for rule in rules if rule != rule_to_remove]
        for rules in rules_by_index
    ]


while not all(rule_by_index):

    for index in range(len(rules_by_index)):
        rules = rules_by_index[index]

        # If there is only one possible rule at a given
        # index then it must fit here. Take the rule and
        # insert it as the answer in the rule_by_index
        # list. Then, remove the rule as a possiblity
        # from all other indices.
        if len(rules) == 1:
            rule = rules[0]
            rule_by_index[index] = rule
            rules_by_index = remove_from_possible_rules(
                rules_by_index, rule
            )

In [8]:
# find the product of values in my ticket for rules
# that start with "departure"
my_ticket = tickets[0]

answer = 1

for val, rule in zip(my_ticket.values, rule_by_index):
    if rule.name.startswith("departure"):
        answer *= val

answer

4381476149273